# Library Imports


In [213]:
import pandas as pd
import configparser
import pyodbc
from datetime import datetime

# Loading Datasets


In [214]:
language_codes = pd.read_csv("./datasets/language_codes.csv")

config = configparser.ConfigParser()
config.read("../server/config.ini")

['../server/config.ini']

# Connecting to Database


In [215]:
conn_str = (
    f"DRIVER={config.get('DATABASE', 'DRIVER')};"
    f"DATABASE={config.get('DATABASE', 'DATABASE')};"
    f"UID={config.get('DATABASE', 'USERNAME')};"
    f"PWD={config.get('DATABASE', 'PASSWORD')};"
    f"SERVER={config.get('DATABASE', 'SERVER')};"
    "sslmode=require;"
)
connxn = pyodbc.connect(conn_str, autocommit=True)

# Custom Functions


In [217]:
def load_df_into_table(
        dataframe: pd.DataFrame, 
        table_name: str,
        cursor: pyodbc.Cursor
    ):
    
    column_list = ",".join(list(dataframe.columns))
    qmarks = str("?,"*len(list(dataframe.columns)))[:-1]
    my_query = f"""INSERT INTO {table_name} ({column_list})
    VALUES ({qmarks});"""
    my_tlist = []
    for row in dataframe.values:
        my_tlist.append(tuple(row))
    cursor.executemany(my_query, my_tlist)

# Load Language Codes


In [218]:
t_language_codes = language_codes
t_language_codes = t_language_codes.where(t_language_codes.notna(), None)


t_language_codes = t_language_codes.rename(
    columns = {
        'English':'language_name',
        "alpha2":"language_code_alpha2",
        "alpha3-t":"language_code_alpha3_t",
        "alpha3-b":"language_code_alpha3_b"
    }
)

# def fix_codes(df, att):
#     df[att] = df[att].str.lower()
#     df[att] = df[att].str.strip()
t_language_codes['language_name'] = t_language_codes["language_name"].str.upper()
t_language_codes['language_display_name'] = t_language_codes["language_name"].str.title()


t_language_codes = t_language_codes.drop(['French'], axis=1)
t_language_codes['is_active'] = 1
t_language_codes['created_date'] = datetime.now()
t_language_codes['created_by'] = "ETL Process"
t_language_codes['updated_date'] = datetime.now()
t_language_codes['updated_by'] = "ETL Process"

In [219]:
load_df_into_table(t_language_codes, "public.bharatarchive_api_language", connxn.cursor())